# Variation due to `word2vec`'s random initialisation

 - fairly small at the word analogy task

In [1]:
%cd ~/NetBeansProjects/ExpLosion/
from notebooks.common_imports import *
from gui.output_utils import *

/Volumes/LocalDataHD/m/mm/mmb28/NetBeansProjects/ExpLosion


In [ ]:
shared = {}
def get_exp_ids_for_reruns_of_word2vec(corpus='amazon_grouped-tagged'):
    query_dict = {'use_similarity': 0, 
          'neighbour_strategy':'linear',
          'vectors__dimensionality': 100, 
          'document_features_ev': 'AN+NN', 
          'document_features_tr': 'J+N+AN+NN', 
          'allow_overlap': False,
          'entries_of': None,
#           'vectors__unlabelled_percentage__in': [70,80], # todo remove this
          'vectors__algorithm': 'word2vec',
          'vectors__composer__in': ['Add', 'Mult', 'Left', 'Right'], # todo Verb???
          'vectors__unlabelled': 'wiki',
          'decode_handler': 'SignifiedOnlyFeatureHandler',
          'noise': 0,
          'use_similarity': 0, 
          'k':3,
          'vectors__unlabelled_percentage': 50,
          'labelled':corpus})
    return Experiment.objects.filter(**query_dict)

In [ ]:
ids = get_exp_ids_for_reruns_of_word2vec() # todo change corpus here
print(ids)
data, folds = get_cv_scores_many_experiment(ids)

reps = [str(x) if x >= 0 else 'avg' for x in get_vectors_field(ids, 'rep')]
df = pd.DataFrame({'F1': data,
                   'Composer': get_vectors_field(ids, 'composer'),
                   'cvfold': folds,
                   'Repetition': reps})
df.to_html('tmp.html')
grid = sns.factorplot(x='Composer', y='F1', hue='Repetition', 
                      data=df, kind='bar');
# grid.set(ylim=(.6, .8));
plt.savefig('plot-w2v_random_init_var.pdf', format='pdf', dpi=300)

 - word embeddings are reasonably stable
 - the lines above for Left and Right are (mostly) flat
     - TODO look deeper into this
 - Composition, esp. multiplication, can result in very different phrase vectors